# MultiFunction Library Modeling and Validation 


In [1]:
#Setup
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import Image

from scipy.stats import gaussian_kde

pd.options.mode.chained_assignment = None  # Suppress the warning    

from utils_f4f import CustomEarlyStopping, AA_hotencoding, parent_model

---------------

# MultiFunction Modeling and Validation of individual assays 




In [2]:
# Data

assays_F4F =  pd.read_csv('data/fit4function_library_screens.csv')
print(assays_F4F)
assays = ['Production2', 'HepG2_bind', 'THLE_bind','HepG2_tr' , 'THLE_tr','Liver'];

#Training data 
train_dfs = dict()
for crnt_assay in assays:
    crnt_df = assays_F4F[['AA',crnt_assay]]
    crnt_df['Output'] = crnt_df[crnt_assay]
    train_dfs[crnt_assay] = crnt_df[['AA','Output']]


            AA  Production1  Production2     Liver  HepG2_bind  HepG2_tr  \
0      AAAARFE     0.487790    -0.086382 -0.444430   -2.011627 -0.800518   
1      AAAETST    -1.377736     0.003127 -3.680569   -2.042271      -inf   
2      AAAFDHK    -1.236781    -1.502219 -0.494197   -1.903524      -inf   
3      AAAFEVL    -2.534514    -3.331298 -0.027308   -1.696997      -inf   
4      AAAKDTY    -0.457144     1.485808 -2.042368   -1.956978 -1.963174   
...        ...          ...          ...       ...         ...       ...   
99995  YYVKARD    -0.916975    -1.554667  1.684322    3.245247  5.231389   
99996  YYVRSDK    -4.413469    -5.286344  1.511374        -inf      -inf   
99997  YYVSNSN    -5.114956    -5.878805  1.082656        -inf      -inf   
99998  YYVSPHE    -1.206143    -2.498700 -0.669647        -inf  0.518604   
99999  YYYQEVE    -5.401841    -5.481131 -1.316135        -inf      -inf   

       THLE_bind   THLE_tr  
0      -1.284821 -4.404453  
1      -3.369956 -1.411144  


In [3]:
# Modeling the six functions 
assay_testnames = ['Production', 'HEPG2_b', 'THLE_b','HEPG2_tr' , 'THLE_tr','Liver']

# Initialization
assays_models = dict()
test_dfs = dict()
intest_dfs = dict()

# Training parameters 
batch_size = 500
#EpochCount = 150
EpochCount = 20 #Used for quick excusion of the Notebook; please use a higher number instead to allow proper convergence


# Iterate over assays 
for crntAssay, cnrt_assay_testname in zip(assays,assay_testnames): 
    print('Currently training: ', crntAssay)
    
    #------------------------------------    
    # Train-Test split
    crnt_df = train_dfs[crntAssay]   
    remove = np.isnan(crnt_df.Output) | np.isinf(crnt_df.Output)
    crnt_df = crnt_df[~remove]

    train_size = int(0.8*len(crnt_df))
    validation_size = int(0.1*len(crnt_df))
    train, validate, test = np.split(crnt_df.sample(frac=1), 
                                     [train_size, train_size+validation_size])
    train = train.reset_index(drop = True)
    validate = validate.reset_index(drop = True)
    test = test.reset_index(drop = True)
    
    #------------------------------------
    # Encoding 
    train_x =  np.asarray([AA_hotencoding(variant) for variant in train['AA']])
    train_y = np.array(list(train.Output))

    validate_x = np.asarray([AA_hotencoding(variant) for variant in validate['AA']])
    validate_y = np.array(list(validate.Output))

    test_x = np.asarray([AA_hotencoding(variant) for variant in test['AA']])
    test_y = np.array(list(test.Output))

    #------------------------------------
    # Fit model
    model = parent_model()
    model.fit(train_x, train_y, batch_size=batch_size, epochs=EpochCount, 
              validation_data=(validate_x, validate_y),verbose=2,
              callbacks=[CustomEarlyStopping(ratio=0.85, patience=3, restore_best_weights = True)])

    
    #------------------------------------
    # Save model
    ModelFileName = 'fit4function_models/fit4function_model6_fxn_'+crntAssay
    model.save(ModelFileName+'.h5')   
    assays_models[crntAssay] = model 
    
    #------------------------------------  
    # Test model 
    # Measured
    x = np.array(test_y)
    # Predicted
    y = model.predict(test_x)
    y = np.reshape(y, (1, y.shape[0]))[0]

    #------------------------------------      
    # Record performance 
    crnt_test = pd.DataFrame([x,y]).transpose()
    crnt_test.columns =  ['Measured','Predicted']
    intest_dfs[cnrt_assay_testname] = crnt_test
    

Currently training:  Production2
Epoch 1/20
158/158 - 4s - loss: 2.5191 - mae: 1.2157 - val_loss: 2.0796 - val_mae: 1.0756 - 4s/epoch - 28ms/step
Epoch 2/20
158/158 - 3s - loss: 1.9798 - mae: 1.0630 - val_loss: 1.9065 - val_mae: 1.0204 - 3s/epoch - 22ms/step
Epoch 3/20
158/158 - 4s - loss: 1.8464 - mae: 1.0208 - val_loss: 1.8160 - val_mae: 1.0211 - 4s/epoch - 23ms/step
Epoch 4/20
158/158 - 4s - loss: 1.7496 - mae: 0.9875 - val_loss: 1.7880 - val_mae: 1.0131 - 4s/epoch - 25ms/step
Epoch 5/20
158/158 - 4s - loss: 1.6894 - mae: 0.9655 - val_loss: 1.6948 - val_mae: 0.9480 - 4s/epoch - 26ms/step
Epoch 6/20
158/158 - 4s - loss: 1.6529 - mae: 0.9524 - val_loss: 1.6631 - val_mae: 0.9414 - 4s/epoch - 27ms/step
Epoch 7/20
158/158 - 4s - loss: 1.6300 - mae: 0.9450 - val_loss: 1.6362 - val_mae: 0.9306 - 4s/epoch - 25ms/step
Epoch 8/20
158/158 - 4s - loss: 1.5951 - mae: 0.9335 - val_loss: 1.6337 - val_mae: 0.9313 - 4s/epoch - 28ms/step
Epoch 9/20
158/158 - 4s - loss: 1.5778 - mae: 0.9281 - val_loss

/Users/slou/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


309/309 [==============================] - 1s 2ms/step
Currently training:  HepG2_bind
Epoch 1/20
123/123 - 4s - loss: 2.6899 - mae: 1.2028 - val_loss: 2.0762 - val_mae: 1.0411 - 4s/epoch - 32ms/step
Epoch 2/20
123/123 - 3s - loss: 1.9350 - mae: 1.0118 - val_loss: 1.7914 - val_mae: 0.9861 - 3s/epoch - 23ms/step
Epoch 3/20
123/123 - 3s - loss: 1.6962 - mae: 0.9562 - val_loss: 1.6097 - val_mae: 0.9371 - 3s/epoch - 24ms/step
Epoch 4/20
123/123 - 3s - loss: 1.5821 - mae: 0.9288 - val_loss: 1.5781 - val_mae: 0.9272 - 3s/epoch - 24ms/step
Epoch 5/20
123/123 - 3s - loss: 1.5231 - mae: 0.9143 - val_loss: 1.4790 - val_mae: 0.9045 - 3s/epoch - 25ms/step
Epoch 6/20
123/123 - 3s - loss: 1.4781 - mae: 0.9034 - val_loss: 1.4707 - val_mae: 0.9047 - 3s/epoch - 25ms/step
Epoch 7/20
123/123 - 3s - loss: 1.4403 - mae: 0.8923 - val_loss: 1.4802 - val_mae: 0.9110 - 3s/epoch - 25ms/step
Epoch 8/20
123/123 - 3s - loss: 1.4359 - mae: 0.8914 - val_loss: 1.4818 - val_mae: 0.9088 - 3s/epoch - 26ms/step
Epoch 9/2

/Users/slou/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 1s 2ms/step
Currently training:  THLE_bind
Epoch 1/20
147/147 - 5s - loss: 1.7684 - mae: 0.9764 - val_loss: 1.5062 - val_mae: 0.8973 - 5s/epoch - 33ms/step
Epoch 2/20
147/147 - 4s - loss: 1.4634 - mae: 0.8626 - val_loss: 1.3446 - val_mae: 0.8134 - 4s/epoch - 24ms/step
Epoch 3/20
147/147 - 4s - loss: 1.3091 - mae: 0.8006 - val_loss: 1.2136 - val_mae: 0.7803 - 4s/epoch - 26ms/step
Epoch 4/20
147/147 - 4s - loss: 1.2464 - mae: 0.7835 - val_loss: 1.1764 - val_mae: 0.7679 - 4s/epoch - 27ms/step
Epoch 5/20
147/147 - 4s - loss: 1.1851 - mae: 0.7649 - val_loss: 1.1293 - val_mae: 0.7524 - 4s/epoch - 24ms/step
Epoch 6/20
147/147 - 4s - loss: 1.1541 - mae: 0.7569 - val_loss: 1.1123 - val_mae: 0.7479 - 4s/epoch - 25ms/step
Epoch 7/20
147/147 - 4s - loss: 1.1337 - mae: 0.7519 - val_loss: 1.0938 - val_mae: 0.7381 - 4s/epoch - 29ms/step
Epoch 8/20
147/147 - 4s - loss: 1.1127 - mae: 0.7458 - val_loss: 1.0900 - val_mae: 0.7434 - 4s/epoch - 29ms/step
Epoch 9/20

...

103/103 [==============================] - 0s 2ms/step
Currently training:  THLE_tr
Epoch 1/20
87/87 - 3s - loss: 3.2851 - mae: 1.4307 - val_loss: 2.7122 - val_mae: 1.2723 - 3s/epoch - 38ms/step
Epoch 2/20
87/87 - 2s - loss: 2.4398 - mae: 1.1957 - val_loss: 2.4645 - val_mae: 1.1847 - 2s/epoch - 26ms/step
Epoch 3/20
87/87 - 2s - loss: 2.2220 - mae: 1.1249 - val_loss: 2.2661 - val_mae: 1.1374 - 2s/epoch - 27ms/step
Epoch 4/20
87/87 - 2s - loss: 2.1190 - mae: 1.0942 - val_loss: 2.1607 - val_mae: 1.1068 - 2s/epoch - 25ms/step
Epoch 5/20
87/87 - 2s - loss: 2.0414 - mae: 1.0742 - val_loss: 2.1319 - val_mae: 1.0907 - 2s/epoch - 25ms/step
Epoch 6/20
87/87 - 2s - loss: 2.0437 - mae: 1.0770 - val_loss: 2.0910 - val_mae: 1.0832 - 2s/epoch - 28ms/step
Epoch 7/20
87/87 - 2s - loss: 1.9930 - mae: 1.0614 - val_loss: 2.0711 - val_mae: 1.0764 - 2s/epoch - 25ms/step
Epoch 8/20
87/87 - 2s - loss: 1.9805 - mae: 1.0588 - val_loss: 2.0926 - val_mae: 1.0925 - 2s/epoch - 26ms/step
Epoch 9/20
87/87 - 2s - loss

/Users/slou/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


170/170 [==============================] - 0s 1ms/step
Currently training:  Liver
Epoch 1/20
158/158 - 5s - loss: 1.3402 - mae: 0.9128 - val_loss: 0.9443 - val_mae: 0.7577 - 5s/epoch - 30ms/step
Epoch 2/20
158/158 - 4s - loss: 0.8584 - mae: 0.7047 - val_loss: 0.8135 - val_mae: 0.6910 - 4s/epoch - 23ms/step
Epoch 3/20
158/158 - 4s - loss: 0.7691 - mae: 0.6551 - val_loss: 0.7501 - val_mae: 0.6417 - 4s/epoch - 23ms/step
Epoch 4/20
158/158 - 3s - loss: 0.7412 - mae: 0.6392 - val_loss: 0.7276 - val_mae: 0.6326 - 3s/epoch - 22ms/step
Epoch 5/20
158/158 - 4s - loss: 0.7252 - mae: 0.6297 - val_loss: 0.7136 - val_mae: 0.6222 - 4s/epoch - 22ms/step
Epoch 6/20
158/158 - 4s - loss: 0.7062 - mae: 0.6195 - val_loss: 0.7258 - val_mae: 0.6284 - 4s/epoch - 23ms/step
Epoch 7/20
158/158 - 4s - loss: 0.6987 - mae: 0.6152 - val_loss: 0.7020 - val_mae: 0.6156 - 4s/epoch - 23ms/step
Epoch 8/20
158/158 - 4s - loss: 0.6907 - mae: 0.6107 - val_loss: 0.6974 - val_mae: 0.6192 - 4s/epoch - 23ms/step
Epoch 9/20
158

/Users/slou/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


309/309 [==============================] - 1s 1ms/step


In [4]:
# Loading Performance from file 
# When plotting from Source Data Files for obtaining the manuscript figures

Predictions = pd.ExcelFile('Source Data Files/Figure 4a.xlsx')
assay_testnames = ['Production', 'HEPG2_b', 'THLE_b','HEPG2_tr' , 'THLE_tr','Liver'];

intest_dfs = dict()
for crnt_assay in assay_testnames:
    intest_dfs[crnt_assay] =pd.read_excel(Predictions, crnt_assay)

FileNotFoundError: [Errno 2] No such file or directory: 'Source Data Files/Figure 4a.xlsx'

In [ ]:
# Figure
assays = assay_testnames.copy()

titles = ['Production',
    'HEPG2 Binding',
    'HEPG2 Transduction',
    'THLE Binding',
    'THLE Transduction',
    'Liver Biodistribution']


# Figure Configurations
sns.set_theme(style='ticks', font_scale=0.75, rc={
    #'font.family': 'sans-serif',
    'font.family': 'Arial',
    #'font.sans-serif': ['Arial', 'DejaVu Sans'],
    'svg.fonttype': 'none',
    'text.usetex': False,
    'pdf.fonttype': 42,
    'ps.fonttype': 42,
    'font.size': 9,
    'axes.labelsize': 9,
    'axes.titlesize': 9,
    'axes.labelpad': 2,
    'axes.linewidth': 0.5,
    'axes.titlepad': 4,
    'lines.linewidth': 0.5,
    'legend.fontsize': 9,
    'legend.title_fontsize': 9,
    'xtick.labelsize': 7,
    'ytick.labelsize': 7,
    'xtick.major.size': 2,
    'xtick.major.pad': 1,
    'xtick.major.width': 0.5,
    'ytick.major.size': 2,
    'ytick.major.pad': 1,
    'ytick.major.width': 0.5,
    'xtick.minor.size': 2,
    'xtick.minor.pad': 1,
    'xtick.minor.width': 0.5,
    'ytick.minor.size': 2,
    'ytick.minor.pad': 1,
    'ytick.minor.width': 0.5,
})

xlims = [
    (-10.5, 8.5),
    (-9.5, 9.5),
    (-10.5, 7.5),
    (-6.5, 7),
    (-9, 7.5),
    (-8, 4.5),
]

fig = plt.figure(figsize=(6, 1.5), dpi=150)
leftpad = 0.12
rightpad = 0.03
wspan = (1-(leftpad)-(rightpad)) / 6
padding = wspan*0.12


# Iterate 
for i, assay in enumerate(assays):
    
    # Configuration of subplots 
    gs = fig.add_gridspec(1, 1, wspace=0.0,
        right=leftpad + ((i+1)*wspan)-padding, left=leftpad + (i*wspan)+padding
    )
    
    ax = fig.add_subplot(gs[0, 0])
    
    # Data 
    df = intest_dfs[assay]    
    x = df['Measured']
    y = df['Predicted']
    
    # Preprocessing 
    remove = (np.isnan(x) | np.isnan(y) | np.isinf(x) | np.isinf(y))
    x = x[~remove]
    y = y[~remove]
    
    
    # Kernels
    kernel = gaussian_kde(np.vstack([
        x.sample(n=1000, random_state=1),
        y.sample(n=1000, random_state=1)
    ]))
    c = kernel(np.vstack([x, y]))
    
    ax.scatter(
        x, y, c=c, s=0.5, cmap=mpl.cm.inferno,
        rasterized=True, linewidth=0, edgecolors=None
    )
     
    xlim = xlims[i]
    ylim = xlims[i]
    
    ax.plot(xlim, xlim, '-r', linewidth=0.3)
    
    
    # Labeling 
    xticks = np.arange(
        np.round((xlim[0] // 2)*2),
        np.round((xlim[1] // 2)*2) + 1, 2)
    ax.set_xticks(xticks) 
    ax.set_yticks(xticks)
    
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(4))
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(2))
    
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(4))
    ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(2))
    
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_aspect('equal', 'box')
    
    ax.set_xlabel('Measured')
    if i == 0:
        ax.set_ylabel('Predicted')
        
    
    # Correlation text
    ax.text(
        x=0.96, y=0.03, transform=ax.transAxes, ha='right', va='bottom',
        s=r'$r$ = {:.2f}'.format(np.corrcoef(x, y)[0, 1]), fontsize=7
    )
    ax.set_title(assay)
    
    
# Show and save 
filename = 'figures/fig4a_multifunction_modeling'
fig.savefig('{}.png'.format(filename))
fig.savefig('{}_600dpi.svg'.format(filename), dpi = 600)
fig.savefig('{}_1200dpi.svg'.format(filename), dpi= 1200)

plt.show()

------
# MultiFunction Distributions 

The distribution of enrichment across variants sampled from the Uniform (3K), Fit4Function (10K), Positive Control (Fit4Function variants satisfying the six conditions), and MultiFunction libraries. The histograms are density-normalized, including non-detected variants (ND).



In [ ]:
# Data and Meta data
df = pd.read_csv('data/multifunction_library.csv')

# Meta data
dataset_labels = [
    'Uniform', 
    'Fit4Function',
    'Positive Control',
    'MultiFunction']
datasets = dataset_labels;

cols = [
    'Production', 
    'HepG2_bind', 'HepG2_tr',
    'THLE_bind', 'THLE_tr',
    'Liver']


In [ ]:
# Figure Configurations

xlims = [
    (-8.5, 4.5), (-8, 6),
    (-8, 4.5), (-8, 6.5),
    (-8, 6), (-6, 3),
]


# colors = ['blue', 'orange', 'green', 'purple']
colors = [
    '#B3B3B3',
    '#FF5E66',
    '#FFA8AE',
    '#488ABA',
]

sns.set_theme(style='ticks', font_scale=0.75, rc={
    'font.family': 'sans-serif',
    # 'font.sans-serif': ['Arial', 'DejaVu Sans'],
    'svg.fonttype': 'none',
    'text.usetex': False,
    'pdf.fonttype': 42,
    'ps.fonttype': 42,
    'font.size': 9,
    'axes.labelsize': 9,
    'axes.titlesize': 9,
    'axes.labelpad': 2,
    'axes.linewidth': 0.5,
    'axes.titlepad': 4,
    'lines.linewidth': 0.5,
    'legend.fontsize': 9,
    'legend.title_fontsize': 9,
    'xtick.major.size': 2,
    'xtick.major.pad': 2,
    'xtick.major.width': 0.5,
    'ytick.major.size': 2,
    'ytick.major.pad': 2,
    'ytick.major.width': 0.5,
    'xtick.minor.size': 2,
    'xtick.minor.pad': 2,
    'xtick.minor.width': 0.5,
    'ytick.minor.size': 2,
    'ytick.minor.pad': 2,
    'ytick.minor.width': 0.5,
})


In [ ]:
# Plot

# More configurations 
fig = plt.figure(figsize=(6, 2), dpi=150)

leftpad = 0.16
rightpad = 0.03
wspan = (1-(leftpad)-(rightpad)) / len(cols)
padding = wspan*0.12

# Iterate over assays 
for c, col in enumerate(cols):
    
    # Subplot configurations 
    gs = fig.add_gridspec(len(datasets), 1, hspace=-0.8, wspace=0.0,
                          # width_ratios=[2, 1], 
                          bottom=0.15,
                          left=leftpad + (c*wspan)+padding, right=leftpad + ((c+1)*wspan)-padding)
    xlim = xlims[c]
    xticks = np.arange(
        np.round((xlim[0] // 2)*2),
        np.round((xlim[1] // 2)*2) + 1,
        2
    )
    bins = np.linspace(xlim[0], xlim[1], 60)
    bins = np.append(-100, bins)

    # Iterate over the four sets 
    for i, dataset in enumerate(datasets):  
        
        # Histogram
        ax = fig.add_subplot(gs[i, 0])
        x = df.loc[df['Label'] == dataset, col]
        nd = (x <= 1e-5)
        x[x <= 0] = 1e-5
        x = np.log2(x)
        freq,_,_ = ax.hist(x, bins=bins, color=colors[i], edgecolor='none', 
                rasterized=True, density=True, alpha=0.6)
        ax.step(bins[1:], freq, color='#888')

        if c == 0:
            ax.text(-0.2, 0, dataset_labels[i], color=colors[i], fontsize=9,
                    ha='right', va='bottom', transform=ax.transAxes)
        
        if c == len(cols)-1 and i == 0:
            ax.yaxis.set_label_position('right')
            ax.set_ylabel('Density', loc='bottom', rotation=270, labelpad=12.)

        # Transparent background and coloring 
        ax.patch.set_alpha(0)

        ax.set_xticks(xticks)
        if i == i == len(datasets) - 1:
            ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(4))
            ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(2))
        else:
            ax.xaxis.set_major_locator(mpl.ticker.NullLocator())
            ax.xaxis.set_minor_locator(mpl.ticker.NullLocator())
        ax.set_yticks([])
        ax.set_xlim(xlim)
        ax.set_ylim([0, 1.0])

        # ['left', 'right', 'bottom', 'top']
        for spine in ax.spines.keys():
            ax.spines[spine].set_visible(False)
            
        ax.spines['bottom'].set_visible(True)
        ax.spines['bottom'].set_linewidth(0.5)
        ax.spines['bottom'].set_color('#CCC')

        # Labeling 
        if i == len(datasets) - 1:
            ax.spines['bottom'].set_color('#444')
            ax.set_xlabel(col)            
        else:
            ax.set_xticks([])
            ax.set_xticklabels([])
            
        rect_width = 0.15
        rect_height = 0.2
        ax.add_patch(mpl.patches.Rectangle(
            ((-1 * rect_width) - 0.02, 0), rect_width, rect_height,
            transform=ax.transAxes, clip_on=False, facecolor=colors[i], alpha=0.5
        ))
        ax.text((-1 * (rect_width / 2))-0.01, rect_height/2, '{:.1f}'.format((nd.sum() / len(x)) * 100), 
                fontsize=5.5, transform=ax.transAxes, rotation=90, ha='center', va='center', color='#444')
        if i == 3:
            ax.text((-1 * (rect_width / 2))-0.02, -0.04, 'ND', transform=ax.transAxes, ha='center', va='top',
                   fontsize=6)
            
# Show and save  
filename = 'figures/fig4b_multifunction_distributions' 
fig.savefig(filename + '.png', transparent=True, dpi=200)
fig.savefig('{}_600dpi.svg'.format(filename), dpi = 600)
fig.savefig('{}_1200dpi.svg'.format(filename), dpi= 1200)

plt.show()
